In [10]:
import html
import pandas as pd
import plotly.express as px
import dash_mantine_components as dmc
import plotly.graph_objects as go
from dash import html, dcc
import pandas as pd
from dash_iconify import DashIconify

In [21]:
def __return_ticktext(plot_df):
    return plot_df.apply(
        lambda x: f"{f'{x.id:0.0f},':<5} {x.description} ({x.unit})", axis=1
    )


def __return_trace(df: pd.DataFrame, color, linestyle, symbol):
    return go.Scatter(
        x=pd.Series(df.value),
        y=pd.Series(df.section_order),
        marker=dict(size=16, color=color, symbol=symbol),
        mode="lines+markers",
        line=go.scatter.Line(color=color, dash=linestyle, width=2),
        connectgaps=True,
    )


def return_section_figure(df: pd.DataFrame, section_id: int):

    df_per_section = df[df["section_id"] == section_id]

    labelmargin = 200

    fig = px.scatter()
    ticktext = __return_ticktext(
        df_per_section[["id", "description", "unit"]].drop_duplicates().reset_index())

    fig.update_layout(
        width=1000,
        height=30 * len(ticktext) + 50,
        xaxis=dict(
            constrain="domain",
            gridcolor="black",
            linecolor="black",
            linewidth=2,
            minor=dict(dtick="L1", tick0="-0.5", gridcolor="black"),
            mirror=False,
            range=[0.5, 19.5],
            showgrid=False,
            showline=False,
            showticklabels=True,
            tickfont=dict(family="Arial", color="black", size=14),
            ticks="outside",
            tickvals=[2, 4, 6, 8, 10, 12, 14, 16, 18],
            ticktext=["1", "2", "3", "4", "5", "6", "7", "8", "9"],
            title="Dezil",
            zeroline=False,
        ),
        yaxis=dict(
            anchor="free",
            constrain="domain",
            gridcolor="black",
            minor=dict(dtick="L1", tick0="-0.5", gridcolor="black"),
            mirror=True,
            range=[len(ticktext) - 0.5, -0.5],
            scaleanchor="x",
            scaleratio=1,
            shift=-200,
            showgrid=False,
            showline=True,
            showticklabels=True,
            side="right",
            title=None,
            zeroline=False,
        ),
        autosize=False,
        margin=dict(autoexpand=False, l=labelmargin, r=0, t=0, b=50),
        showlegend=False,
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)"
    )

    fig.add_shape(
        # Rectangle with reference to the plot
        type="rect",
        xref="x domain",
        yref="y domain",
        x0=0,
        y0=0,
        x1=1.0,
        y1=1.0,
        line=dict(
            color="black",
            width=1,
        ),
    )

    fig.update_layout(
        yaxis=dict(
            tickfont=dict(family="Arial", color="black", size=14),
            tickmode="array",
            ticktext=ticktext,
            tickvals=ticktext.index,
        )
    )

    for file_id in df_per_section["file_id"].values:
        for hand in df_per_section["hand"].values:
            color = px.colors.qualitative.G10[file_id]
            linestyle = "solid" if hand == "right" else "dash"
            symbol = "circle" if hand == "right" else "diamond-open"

            in_df = df_per_section.set_index(
                ["file_id", "hand"]).sort_index().loc[(file_id, hand)]
            fig.add_trace(__return_trace(in_df, color, linestyle, symbol))

    return fig


def wrap_figure_in_graph(title: str, figure):
    return html.Div(
        [
            dmc.Title(title, order=2),
            dcc.Graph(
                # id="_wait_time_graph",
                style={"height": "100%", "width": "100%"},
                className="wait_time_graph",
                config={
                    "staticPlot": False,
                    "editable": False,
                    "displayModeBar": False,
                },
                figure=figure,
            )
        ],
        style={
            # "padding-left": "20px",
        }
    )

In [22]:
input = pd.read_csv("plot_input.csv")

In [23]:
section_id = 0
df = input
return_section_figure(df,0)

In [24]:
return_section_figure(df,1)

In [20]:
df_per_section = df[df["section_id"]==1]
df_per_section[["id", "description", "unit"]].drop_duplicates()

,id,description,unit
2,3,Fingerlänge,mm
